In [2]:
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.utils import make_grid
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm.notebook import tqdm
from IPython.display import clear_output

In [3]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        
        self.downsample = downsample
        self.relu = nn.ReLU()
        
        self.cl1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        
        self.cl2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels)
        )
        
    def forward(self, x):
        identity = x
        x = self.cl1(x)
        x = self.cl2(x)
        if self.downsample:
            identity = self.downsample(identity)
        x = self.relu(x + identity)
        return x
    
    
class ConvolutionalEmbedding(nn.Module):
    def __init__(self, block, layers, in_channels=1, out_channels=64, embed_dim=128):
        super().__init__()
        self.in_planes = 64
        
        self.cl1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        
        self.maxpool1 = nn.MaxPool2d(3, 2, 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.maxpool2 = nn.MaxPool2d(4, stride=2)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2*512, embed_dim)
        )
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes)
            )
        layers = []
        layers.append(block(self.in_planes, planes, stride, downsample))
        self.in_planes = planes
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)
        
        
    def forward(self, x):
        x = self.cl1(x)
        x = self.maxpool1(x)
        
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.maxpool2(x)

        x = self.fc(x)
        
        return x    

In [3]:
import models.data_loader as dl
from utils import initialize_logging, load_config

config = load_config(config_path="./config/config.yaml")
initialize_logging(config_path="./config/logging_config.yaml", debug=False)

train_loader = DataLoader(
        dl.CoverDataset(data_path='dataset', file_ext='npy', 
                        dataset_path='dataset/train', data_split='train', debug=False, max_len=128),
        batch_size=128,
        num_workers=0,
        shuffle=True,
        drop_last=False
    )

val_loader = DataLoader(
        dl.CoverDataset(data_path='dataset', file_ext='npy', 
                        dataset_path='dataset/train', data_split='val', debug=False, max_len=128),
        batch_size=16,
        num_workers=0,
        shuffle=False,
        drop_last=False
    )

In [7]:
def eval_metrics_embed(anchor_embed, pos_embed, neg_embed):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    if (0.5 - cos(anchor_embed, pos_embed) + cos(anchor_embed, neg_embed)) < 0.0:
        return 1
    return 0

def eval_metrics(anchor_logit, pos_logit, neg_logit):
    return np.sum([eval_metrics_embed(anchor_logit[i], pos_logit[i], neg_logit[i]) 
            for i in range(len(anchor_logit))])

def evaluate(model, data_loader, loss_fn, batch_size):
    loss = 0
    accuracy = 0
    batch_count = 0

    model.eval()

    for i, batch in enumerate(tqdm(data_loader)):
        with torch.no_grad():

            X_anchor = batch['anchor'][:, None, :, :].to(device)
            X_pos = batch['positive'][:, None, :, :].to(device)
            X_neg = batch['negative'][:, None, :, :].to(device)


            anchor_logit = model(X_anchor)
            pos_logit = model(X_pos)
            neg_logit = model(X_neg)

            loss += loss_fn(anchor_logit, pos_logit, neg_logit)
            accuracy += eval_metrics(anchor_logit, pos_logit, neg_logit)
            batch_count += 1

    loss /= batch_count
    accuracy /= (batch_count * batch_size)
    
    return accuracy, loss.item()

In [5]:
def train(model, device):
    optim = torch.optim.Adam(model.parameters(), lr=0.0006)
    loss_fn = nn.TripletMarginWithDistanceLoss(margin=0.5,  swap=False,
                                               distance_function=(lambda x, y: 1.0 - f.cosine_similarity(x, y)))
    n_epochs = 10
    batch_size = 128

    for epoch in range(n_epochs):
        tqdm_loader = tqdm(train_loader)
        for i, batch in enumerate(tqdm_loader):
            model.train()

            X_anchor = batch['anchor'][:, None, :, :].to(device)
            X_pos = batch['positive'][:, None, :, :].to(device)
            X_neg = batch['negative'][:, None, :, :].to(device)

            anchor_logit = model(X_anchor)
            pos_logit = model(X_pos)
            neg_logit = model(X_neg)

            loss = loss_fn(anchor_logit, pos_logit, neg_logit)
            tqdm_loader.set_postfix({"loss":loss.item()})

            optim.zero_grad() 
            loss.backward()
            optim.step()

        print('On epoch end', epoch)
        acc_val_epoch, loss_val_epoch = evaluate(model, val_loader, loss_fn, 16)
        print('Val acc:', acc_val_epoch, 'Val loss:', loss_val_epoch)
        torch.save(model, 'cover_embedding_2.pth')

In [ ]:
#model create + train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embed_dim = 128
model = ConvolutionalEmbedding(Block, layers=[3, 3, 3, 4])
model.to(device)

train(model, device)

In [9]:
#load model + train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embed_dim = 128
model = torch.load('cover_embedding_2.pth')
model.to(device)

train(model, device)

  0%|          | 0/2337 [00:00<?, ?it/s]

On epoch end 0


  0%|          | 0/993 [00:00<?, ?it/s]

Val acc: 0.5450025176233635 Val loss: 0.19344429671764374


  0%|          | 0/2337 [00:00<?, ?it/s]

...

On epoch end 8


  0%|          | 0/993 [00:00<?, ?it/s]

Val acc: 0.6113418932527693 Val loss: 0.14787407219409943


  0%|          | 0/2337 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
import models.data_loader as dl
from utils import initialize_logging, load_config

test_data = dl.CoverDataset(data_path='dataset', dataset_path='dataset/test', file_ext='npy', data_split='test', debug=False, max_len=128)
embed_dim = 128

model = torch.load('cover_embedding_2.pth').to('cuda')

X_id = torch.zeros(len(test_data))
X_embed = torch.zeros((len(test_data), embed_dim))

with torch.no_grad():
    model.eval()
    for i in range(len(test_data)):
        X_id[i] = test_data[i]['anchor_id']
        X = test_data[i]['anchor']
        X = torch.unsqueeze(X, 0)
        X = X[:, None, :, :].to('cuda')
        X_embed[i] = model(X)

In [24]:
#write embeddings to the file
f = open('embeddings.txt', 'w')

for i in range(len(X_id)):
    
    line = str(X_id[i])
    for j in range(len(X_embed[i])):
        line += " " + str(X_embed[i][j])
    line += "\n"
        
    f.write(line)
        
f.close()

In [ ]:
#dist matrix init

cos = nn.CosineSimilarity(dim=0, eps=1e-6)
dists = torch.zeros((X_embed.shape[0], X_embed.shape[0]))
for i in range(X_embed.shape[0]):
    for j in range(i, X_embed.shape[0]):
        dists[i, j] = 1.0 - cos(X_embed[i], X_embed[j])
        dists[j, i] = dists[i, j]
dists = dists.numpy()

dists

In [ ]:
#solution

f = open('solution.txt', 'w')

for i in range(dists.shape[0]):
    dists[i,i] = np.inf
    idx = np.argpartition(dists[i], 100)[:100]
    line = str(X_id[i])
    for j in idx:
        line += " " + str(X_id[j]) 
        
    line += "\n"
    f.write(line)
    
f.close()